In [ ]:
import openpyxl
import datetime
from datetime import datetime as dtime
import datetime
import dateutil.relativedelta
from calendar import monthrange
def read_config():
    bankNamefile="config.xlsx"
    sheetName="bancos"
    wb = openpyxl.load_workbook(bankNamefile)
    sheet = wb[sheetName]
    n=sheet.max_row
    m=sheet.max_column
    listBanks=[x[0].value for x in sheet['A2:A'+str(n)]]
    listFields=[x.value for x in sheet['B1:P1'][0]]
    data={}
    for i,bank in enumerate(listBanks):
        data[bank]={}
        for j,field in enumerate(listFields):
            datacell=sheet.cell(row=i+2, column=j+2).value
            data[bank][field]=datacell
    return data
data=read_config()

def get_sheet(bankName,sheetName):
    bankNamefile=f"formatos/{data[bankName]['NombreArchivo']}"
    sheetName=data[bankName]['NombreHoja']
    #print(bankNamefile,sheetName)
    wb = openpyxl.load_workbook(bankNamefile)
    sheet = wb[sheetName]
    return sheet
def get_dateRow(sheet,i,dateColumn):
    if sheet.cell(row=i, column=dateColumn).value==None:
        dateRow=''
    else:
        dateRow=sheet.cell(row=i, column=dateColumn).value
    return dateRow

def get_documentNr(bankName,sheet,i,documentColumn,codBancaColumn):
    #print("documento :")
    #print(sheet.cell(row=i, column=codBancaColumn).value)
    #print(type(sheet.cell(row=i, column=documentColumn).value))
    if bankName=='MERCANTIL':
        if sheet.cell(row=i, column=documentColumn).value==None or sheet.cell(row=i, column=documentColumn).value=='':
            if sheet.cell(row=i, column=codBancaColumn).value==None:
                nroDocument=''
                #print("acaaaa")
            else:
                nroDocument=sheet.cell(row=i, column=codBancaColumn).value
                #print("allaaaa")
        else:
            nroDocument=sheet.cell(row=i, column=documentColumn).value
            #print("aquiiiii")
    else:
        if sheet.cell(row=i, column=documentColumn).value==None:
            nroDocument=''
        else:
            nroDocument=sheet.cell(row=i, column=documentColumn).value
    return nroDocument
def get_description(bankName,sheet,i,descriptionColumn,nombreColum):
    if bankName=="MERCANTIL":
        if sheet.cell(row=i, column=descriptionColumn).value==None:
            description=''
        else:
            description=sheet.cell(row=i, column=descriptionColumn).value
        if sheet.cell(row=i, column=nombreColum).value==None:
            name=''
        else:
            name=sheet.cell(row=i, column=nombreColum).value
        description=name+"-"+description
    else:   
        if sheet.cell(row=i, column=descriptionColumn).value==None:
            description=''
        else:
            description=sheet.cell(row=i, column=descriptionColumn).value
    return description
def get_saldo(sheet,i,descriptionColumn,amountColumn):
    if sheet.cell(row=i, column=amountColumn).value==None:
        saldo=0
    else:
        saldo=float(str(sheet.cell(row=i, column=amountColumn).value).replace(",",""))
    return saldo
def get_amount(bankName,sheet,i,amountColumn,creditColumn,debitColumn):
    if bankName=="MERCANTIL" or bankName=="FASSIL":
        if sheet.cell(row=i, column=creditColumn).value==None:
            credit=0
        else:
            credit=float(str(sheet.cell(row=i, column=creditColumn).value).replace(",",""))

        if sheet.cell(row=i, column=debitColumn).value==None:
            debit=0
        else:
            debit=float(str(sheet.cell(row=i, column=debitColumn).value).replace(",",""))
        amount=credit-debit
    else:
        if sheet.cell(row=i, column=amountColumn).value==None:
            amount=0
        else:
            amount=float(str(sheet.cell(row=i, column=amountColumn).value).replace(",",""))
    return amount
def get_typetrx(description,amount):
    if description!='':
        if description.find("ITF")!=-1:
            typetrx="ZITF"
        else:
            if amount>0:
                typetrx="Z001"
            else:
                typetrx="Z002"
    else:
        if amount>0:
            typetrx="Z001"
        else:
            typetrx="Z002"
    return typetrx
def read_bank(bankName,bankNamefile):
    sheetName=data[bankName]['NombreHoja']
    sheet = get_sheet(bankName,sheetName)
    dataUnion=[]

    cuentaColumn=data[bankName]['CuentaCol']
    celdaFecha=data[bankName]['CeldaPeriodo']
    dateColumn=ord(celdaFecha[0])-64 #PARA SALIDA 1
    maxRowColumn=data[bankName]['MaxRowCol']
    documentColumn=data[bankName]['NroDocumCol'] #PARA SALIDA 2 

    creditColumn=data[bankName]['CreditCol']#OPCIONAL PARA SALIDA 3
    debitColumn=data[bankName]['DebitCol'] #OPCIONAL PARA SALIDA 3
    importColumn=data[bankName]['ImporteCol'] # ES PARA SALIDA 3
    saldoColumn=data[bankName]['SaldoCol'] #OPCIONAL PARA SALIDA 3
    nombreColum=data[bankName]['NombreCol']#OPCIONAL PARA SALIDA EN SOLO 1 BANCO 
    descripcionColumn=data[bankName]['DescripcionCol']#OPCIONAL ES PARA SALIDA 4
    codBancaColumn=data[bankName]['CodBancario']#OPCIONAL 
    
    dateformat=data[bankName]['FormatoFecha']
    dateExcel=sheet[celdaFecha].value
    dateCero = datetime.datetime.strptime(str(dateExcel), dateformat)+ dateutil.relativedelta.relativedelta(months=0)
    initialDate=datetime.datetime(dateCero.year,dateCero.month,1)
    finalDate=datetime.datetime(dateCero.year,dateCero.month,monthrange(dateCero.year,dateCero.month)[1])
    i=int(celdaFecha[1:])

    #print(i,sheet.max_row,descripcionColumn)
    while i<=sheet.max_row:
        if sheet.cell(row=i, column=maxRowColumn).value!=None:
            dateRow=get_dateRow(sheet,i,dateColumn)
            nroDocument=get_documentNr(bankName,sheet,i,documentColumn,codBancaColumn)
            saldo=get_saldo(sheet,i,saldoColumn,saldoColumn)
            description=get_description(bankName,sheet,i,descripcionColumn,nombreColum) #DINAMICO 
            amount=get_amount(bankName,sheet,i,importColumn,creditColumn,debitColumn) #DINAMICO
            typetrx=get_typetrx(description,amount)
            unionRow={
                "date":dateRow,
                "documentNr":nroDocument,
                "description":description,
                "type":typetrx,
                "amount":amount,
                "saldo":saldo,
            }
            dataUnion.append(unionRow)
        i=i+1
    initialBalance=dataUnion[0]['saldo']-dataUnion[0]['amount']
    finalBalance=dataUnion[len(dataUnion)-1]['saldo']
    #print(initialBalance,finalBalance)
    #print(pd.DataFrame(dataUnion))
    return {'data':dataUnion,'initialBalance':initialBalance,
    'finalBalance':finalBalance,'initialDate':initialDate,
    'finalDate':finalDate,'account':data[bankName]['CuentaCol'],'namComercial':data[bankName]['NombreComercial']}
def make_templates(infobankk):
    binAccount=5385
    dateFname="21062022"
    dateFnameOuput=dtime.today().strftime("%d%m%Y")
    #print(dateFname)
    wb = openpyxl.load_workbook(f"formatos/{binAccount}-{dateFname}.xlsx")
    sheet = wb["UNION"]
    initialDate=infobank['initialDate'].strftime("%d/%m/%Y")
    finalDate=infobank['finalDate'].strftime("%d/%m/%Y")
    initialBalance=infobank['initialBalance']
    finalBalance=infobank['finalBalance']
    accountNumber=infobank['account']
    newPeriod=f"ESTADO DE CUENTA DEL {initialDate} AL {finalDate}"
    dataSap=infobank['data']
    binAccountouput=infobank['account'][:4]
    sheet["A1"]=newPeriod
    sheet["C4"]=accountNumber
    sheet["A9"]=initialBalance
    sheet["B9"]=finalBalance
    sheet["A8"]=initialDate
    sheet["B8"]=finalDate
    sheet["B3"]=infobank['namComercial']
    rowInit=17
    for i in range(len(dataSap)):
        sheet.cell(row=rowInit,column=1).value=dataSap[i]['date']
        sheet.cell(row=rowInit,column=2).value=dataSap[i]['documentNr']
        sheet.cell(row=rowInit,column=3).value=dataSap[i]['description']
        sheet.cell(row=rowInit,column=4).value=dataSap[i]['type']
        sheet.cell(row=rowInit,column=5).value=dataSap[i]['amount']
        rowInit=rowInit+1
    wb.save(f"Plantillas/{binAccountouput}-{dateFnameOuput}.xlsx")

for bk in list(data.keys()):
    print(bk,data[bk]['NombreArchivo'])
    infobank=read_bank(bk,data[bk]['NombreArchivo'])
    make_templates(infobank)
#infobank=read_bank("ECONOMICO","banco economico.xlsx","ECONOMICO")
#make_templates(infobank)

In [ ]:
from pathManagement import (outPath)
print(outPath)


In [5]:
import pandas as pd                                 
cuentas=pd.read_excel("config.xlsx",sheet_name="cuentas").to_dict('records')
len(cuentas)

36

In [8]:
import os
res = []
dir_path = r'C:\Users\x11832\Documents\ProyectosDaniel\VENADO\sapHunter\SAPHunter\extractosBancarios\16102022'
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path[-4:])

print(res)



['0014-16102022.', '0210-16102022.', '0635-16102022.', '1539-16102022.', '5385-16102022.', '6211-16102022.']
